In [2]:
#讀取文本轉成分割檔
import re
import os
import csv
import pandas as pd

def split_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    # 使用正規表達式進行分割（考慮句點、空白和換行符號）
    #pattern = re.compile(r'(\.\s|\n)(?![^\(]*\))(?!(?<=Dr\.\s))(?![^()]*\))(?!(?<=D\.\s))')
    pattern = re.compile(r'(\.\s|\n)(?![^\(]*\))(?![^()]*\))(?!(?<=D\.\s))(?!(?<=Dr\.\s)[^\s]*\s)')
    sentences = re.split(pattern, content)
    sentences = [item for item in sentences if item is not None]
    output = []

    current_position = 0
    for i in range(0, len(sentences)-1, 2):  # 修改範圍，避免超出列表範圍
        sentence = sentences[i] + sentences[i + 1]  # 合併句點和後面的空白或換行符號
        if sentence.strip() or sentence == '\n':  # 跳過空白句子
            start_position = current_position  # 起始位置改為目前位置
            end_position = start_position + len(sentence)
            output.append((start_position, end_position, sentence))
            current_position = end_position

    return output



def print_output_to_dataframe(output):
    input_file_name = os.path.splitext(os.path.basename(file_name))[0]
    input_data = {'InputFile': [], 'LineNumber': [], 'Paragraph': []}

    for index, entry in enumerate(output, start=1):
        paragraph = entry[2]
        if paragraph != '':
            input_data['InputFile'].append(input_file_name)
            input_data['LineNumber'].append(entry[0])
            input_data['Paragraph'].append(paragraph)
    df = pd.DataFrame(input_data)
    return df




folder_path = "/content/drive/MyDrive/opendid_test"
files = os.listdir(folder_path)
sorted_files = sorted(files)

result_df = pd.DataFrame(columns=['InputFile', 'LineNumber', 'Paragraph'])

for file_name in sorted_files:
    file_path = os.path.join(folder_path, file_name)
    result = split_text(file_path)
    result_df = pd.concat([result_df, print_output_to_dataframe(result)], ignore_index=True)



# 匯出 DataFrame 到 'Split.tsv' 檔案
print(result_df)
result_df['Paragraph'] = result_df['Paragraph'].str.replace('\t',' ')
result_df['Paragraph'] = result_df['Paragraph'].str.replace('\n',' ')
result_df.to_csv('Split.tsv', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\', doublequote=False)






       InputFile LineNumber                                    Paragraph
0           1097          0                                 433475.RDC\n
1           1097         12                             Timmins, ELDEN\n
2           1097         27                          43J47561,43J47561\n
3           1097         45                                           \n
4           1097         46          Last edited : 7/9/2063\t\tPage: 2\n
...          ...        ...                                          ...
90258  file66968       3845                                           \n
90259  file66968       3846                                          C. 
90260  file66968       3849  OMENTUM - NEGATIVE FOR TUMOUR - SEE TEXT.\n
90261  file66968       3891                                           \n
90262  file66968       3892                               8/10/2069 km\n

[90263 rows x 3 columns]


In [ ]:
#訓練用刪除NULL
file_path = 'Split.txt'
output_path = 'answerNA.txt'

with open(file_path, 'r') as input_file, open(output_path, 'w') as output_file:
    for line in input_file:
        # 移除行尾的換行符
        line = line.strip()
        # 分割資料行
        data = line.split('\t')
        #print(data)
        # 檢查資料行是否缺少最後一項
        if (data[3:4] == ['PHI: NULL']):
          #print(f"Error in line }: {line}")
          print("0")
        else:
          output_file.write(line+'\n')


In [3]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-160m"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
from datasets import load_dataset, Features, Value
dataset = load_dataset("csv", data_files="aaa.tsv", delimiter='\t',
                       features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                       column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [101]:
import torch
sub_datasets = torch.utils.data.random_split(dataset['train'], [100003 ,0])
print(len(sub_datasets[0]))
for i in range(4): print(sub_datasets[0][i])

100003
{'fid': '361', 'idx': 1907, 'content': 'MICROSCOPIC: (Reported by Dr H Nicewander)', 'label': 'DOCTOR: H Nicewander'}
{'fid': '34', 'idx': 1102, 'content': 'FROZEN SECTION REPORT:', 'label': 'PHI: NULL'}
{'fid': 'file16922', 'idx': 2512, 'content': 'C.', 'label': 'PHI: NULL'}
{'fid': 'file11031', 'idx': 204, 'content': 'OMNI-Lab_11', 'label': 'PHI: NULL'}


In [103]:
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
PAD_IDX
IGNORED_PAD_IDX = -100
PAD_IDX

50278

In [104]:
from torch.utils.data import DataLoader
import torch

train_data = list(sub_datasets[0])

def collate_batch(batch):
    texts = [f"{bos} {data['content']} {sep}"+ data['label'].replace('\\n','\n')+f" {eos}" for data in list(batch)] # 範例 prompt
    encoded_seq = tokenizer(texts, padding=True)

    indexed_tks = torch.tensor(encoded_seq['input_ids'])
    attention_mask = torch.tensor(encoded_seq['attention_mask'])
    encoded_label = torch.tensor(encoded_seq['input_ids'])
    encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

    return indexed_tks, encoded_label, attention_mask

train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=collate_batch)
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
print(tks)
next(iter(titer))

torch.Size([2, 28])
tensor([[    0, 16432,  1311,  2697, 27004,  1864,    27,   313,  6802,  7551,
           407,  3196,   388, 29902,    88,  5945,    10,   209, 50279, 13220,
         39321,    27,   388, 29902,    88,  5945,   209, 50277],
        [50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278, 50278,
             0,  7499,    48,    59,  1400,  6725,  8286,  5689, 13766,    27,
           209, 50279,  6663,    42,    27,  5812,   209, 50277]])


(tensor([[50278, 50278, 50278, 50278,     0,   330,    15,   209, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0, 37503, 13652,    14, 21663,    64,   883,   209, 50279,  6663,
             42,    27,  5812,   209, 50277]]),
 tensor([[ -100,  -100,  -100,  -100,     0,   330,    15,   209, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0, 37503, 13652,    14, 21663,    64,   883,   209, 50279,  6663,
             42,    27,  5812,   209, 50277]]),
 tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [105]:
import random
BATCH_SIZE = 5 # 自行決定大小

class BatchSampler():
    def __init__(self, data, batch_size):
        self.pooled_indices = []
        self.data = data
        self.batch_size = batch_size
        self.len = len(list(data))
    def __iter__(self):
        self.pooled_indices = []
        indices = [(index, len(data["content"])) for index, data in enumerate(self.data)]
        random.shuffle(indices)
        for i in range(0, len(indices), BATCH_SIZE * 100):
            self.pooled_indices.extend(sorted(indices[i:i + BATCH_SIZE * 100], key=lambda x: x[1], reverse=True))
        self.pooled_indices = [x[0] for x in self.pooled_indices]

        for i in range(0, len(self.pooled_indices), BATCH_SIZE):
            yield self.pooled_indices[i:i + BATCH_SIZE]
    def __len__(self):
        return (self.len + self.batch_size - 1) // self.batch_size

bucket_train_dataloader = DataLoader(train_data, batch_sampler=BatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=collate_batch, pin_memory=True)
#for idx,batch in enumerate(bucket_train_dataloader):
  #print(batch)

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
import torch
from tqdm import tqdm, tqdm_notebook
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sample_text(model, tokenizer, seed, n_words=20):
    model = model.to(device)
    model.eval()
    text = tokenizer.encode(seed)
    inputs, past_key_values = torch.tensor([text]), None
    with torch.no_grad():
        for _ in tqdm(range(n_words)):
            out = model(inputs.to(device), past_key_values=past_key_values)
            logits = out.logits
            past_key_values = out.past_key_values
            log_probs = F.softmax(logits[:, -1], dim=-1)
            inputs = torch.multinomial(log_probs, 1)
            text.append(inputs.item())
            if tokenizer.decode(inputs.item()) == eos:
                break


    return tokenizer.decode(text)

sample_text(model, tokenizer, seed=f"{bos} DR AADLAND ABRAHAM {sep}")

100%|██████████| 20/20 [00:07<00:00,  2.73it/s]


"<|endoftext|> DR AADLAND ABRAHAM \n\n####\n\n IN THE TOPBURNS REPORTS alike for virtually everything Disaster Killer, Gabe's"

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 5 # 設定你的訓練次數
optimizer = AdamW(model.parameters(),lr=5e-5)

steps = len(bucket_train_dataloader)
total_steps = steps * EPOCHS
print(steps, total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps*0.1,
    num_training_steps=total_steps
)

model.resize_token_embeddings(len(tokenizer))
model.to(device)
print(f'Total numbers of steps: {total_steps}')
model


20001 100005
Total numbers of steps: 100005


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [ ]:
from tqdm import tqdm,trange

global_step = 0
total_loss = 0

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    predictions , true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch:  20%|██        | 1/5 [27:30<1:50:02, 1650.58s/it]

Average train loss: 1.4753026594329397


Epoch:  40%|████      | 2/5 [55:12<1:22:50, 1656.98s/it]

Average train loss: 1.0946212699085967


Epoch:  60%|██████    | 3/5 [1:23:00<55:24, 1662.34s/it]

Average train loss: 0.9126937505343719


Epoch:  80%|████████  | 4/5 [1:50:54<27:46, 1666.89s/it]

Average train loss: 0.7556457584646069


In [ ]:
# 指定儲存路徑
model_save_path = "/content/drive/MyDrive/model"

# 儲存模型
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

NameError: ignored

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup, AdamW
from tqdm import tqdm
from datasets import load_dataset, Features, Value

In [6]:
# 載入訓練好的模型
pretrained_model_name="/content/drive/MyDrive/epcoh_10"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

HFValidationError: ignored

In [98]:
# 分詞器設定
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'
special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}
tokenizer.add_special_tokens(special_tokens_dict)
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
IGNORED_PAD_IDX = -100
tokenizer.padding_side = 'left'

In [96]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="Split.tsv", delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
valid_list= list(valid_data['train'])
valid_list

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

[{'fid': '1097', 'idx': 0, 'content': ' 433475.RDC ', 'label': None},
 {'fid': '1097', 'idx': 12, 'content': 'Timmins, ELDEN ', 'label': None},
 {'fid': '1097', 'idx': 27, 'content': '43J47561,43J47561 ', 'label': None},
 {'fid': '1097', 'idx': 45, 'content': ' ', 'label': None},
 {'fid': '1097',
  'idx': 46,
  'content': 'Last edited : 7/9/2063  Page: 2 ',
  'label': None},
 {'fid': '1097', 'idx': 78, 'content': 'CLINICAL: ', 'label': None},
 {'fid': '1097',
  'idx': 88,
  'content': 'Metastatic cancer ?colorectal primary. ',
  'label': None},
 {'fid': '1097', 'idx': 127, 'content': ' ', 'label': None},
 {'fid': '1097', 'idx': 128, 'content': 'MACROSCOPIC: ', 'label': None},
 {'fid': '1097',
  'idx': 141,
  'content': 'Specimen labelled "Omentum secondary", consists of a piece of omentum 120 x 100 x 30mm. ',
  'label': None},
 {'fid': '1097',
  'idx': 229,
  'content': ' On sectioning there are multiple fibrotic white ill-defined nodules identified. ',
  'label': None},
 {'fid': '1097

In [106]:
# 使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 768)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=768, out_features=3072, bias=True)
          (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=True)
          

In [107]:
from tqdm import tqdm#, tqdm_notebook

tokenizer.padding_side = 'left'
def sample_batch(model, tokenizer, input):
    """Generate text from a trained model."""
    model.eval()
    seeds = [f"{bos} {text['content']} {sep}" for text in input]
    texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
    outputs = []
    #return
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=400, pad_token_id = PAD_IDX,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        for idx , pred in enumerate(preds):
            pred = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
            if pred == "PHI: NULL":
                continue
            phis = pred.split('\n')
            lidxs = {}
            for p in phis:
                tid = p.find(':')
                if tid > 0:
                    text = p[tid+1:].strip()
                    nv = text.find('=>')
                    normalizedV = None
                    #
                    if nv>0:
                      normalizedV = text[nv+2:]
                      text = text[:nv]


                    #
                    lidx = 0
                    if text in lidxs:
                        lidx = lidxs[text]
                    lidx = input[idx]['content'].find(text, lidx)
                    eidx = lidx+len(text)
                    lidxs[text] = eidx
                    sidx=int(input[idx]['idx'])
                    if normalizedV is None:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}')
                    else:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}\t{normalizedV}')
    return outputs

f = open("answer.txt", "w",encoding="UTF-8")
BATCH_SIZE = 5
for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
        seeds = valid_list[i:i+BATCH_SIZE]
        outputs = sample_batch(model, tokenizer, input=seeds)
       # print(seeds)
        for o in outputs:
            f.write(o)
            f.write('\n')
f.close()

100%|██████████| 3477/3477 [02:42<00:00, 21.42it/s]


In [ ]:
file_path = 'answer.txt'
output_path = 'answer0.txt'

with open(file_path, 'r'，encoding='utf-8') as input_file, open(output_path, 'w'，encoding='utf-8') as output_file:
    for line in input_file:
        # 移除行尾的換行符
        line = line.strip()
        # 分割資料行
        data = line.split('\t')
        # 檢查資料行是否缺少最後一項
        if (len(data) >= 5):
            # 如果有最後一項，則寫入到新的檔案中
            output_file.write(line + '\n')
        else:
          print(f"Error in line {line_number}: {line}")

In [ ]:


def process_data(input_file, output_file):
    with open(input_file, 'r'，encoding='utf-8') as input_file, open(output_file, 'w', encoding='utf-8') as output_file:
        previous_data = set()

        for line in input_file:
            data_identifier = line.split('\t')[:5]
            data_identifier = '\t'.join(data_identifier)

            # 如果這個資料與前一行相同，則跳過這行
            if data_identifier in previous_data:
              #print(line)
              continue

            # 如果資料不同，則將其輸出到輸出檔案中
            output_file.write(line)

            # 更新已經見過的資料集合
            previous_data.add(data_identifier)



A = open("answer1.txt", "w",encoding="ascii")
process_data('answer0.txt', 'answer1.txt')
A.close()

In [ ]:
file_path = 'answer1.txt'
output_path = 'answer2.txt'


with open(file_path, 'r') as input_file, open(output_path, 'w') as output_file:
    for line in input_file:
        # 移除行尾的換行符
        line = line.strip()
        # 分割資料行
        data = line.split('\t')

        # 檢查資料行是否缺少最後一項
        if (len(line) >= 150):
             #如果有重複，跳過
             print(f"Error in line : {line}")
        else:
            output_file.write(line + '\n')



